In [2]:
# dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd


In [3]:
# create database connection
database_path = "postgres:David$1986@localhost:5432/Melbourne_Business_db"
engine = create_engine(f'postgresql://{database_path}')
session = Session(engine)

In [5]:
# view classes use automap found
Base.classes.keys()

[]

## use automap

In [8]:
# reflect database
Base = automap_base()
Base.prepare(engine, reflect=True)
engine.table_names()

c:\Users\david\Anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  after removing the cwd from sys.path.


['melbourne_business']

In [9]:
# view classes use automap
Base.classes.keys()

[]

In [10]:
# Save references to table
Melbourne_Business = Base.classes.melbourne_business

AttributeError: melbourne_business

## Inspecting database

In [9]:
# Inspecting data in table
inspector = inspect(engine)
inspector.get_table_names()

['melbourne_business']

In [10]:
engine.execute('SELECT * FROM melbourne_business').first()

(2002, 'Melbourne (CBD)', '469-479 King Street MELBOURNE 3000', 'Zoological and Botanical Gardens Operation', 'Seats - Indoor', 113, Decimal('144.9595'), Decimal('-37.8206'))

In [11]:
# Inspecting columns in table
columns = inspector.get_columns('melbourne_business')
for c in columns:
    print(c['name'], c["type"])

census_year INTEGER
clue_small_area VARCHAR(100)
trading_name VARCHAR(100)
industry_anzsic4_description VARCHAR(100)
seating_type VARCHAR(100)
number_of_seats INTEGER
x_coordinate NUMERIC
y_coordinate NUMERIC


In [23]:
# reading sql using panda
database_df = pd.read_sql("Select * from melbourne_business",engine)
database_df.head(10)

,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,2002,Melbourne (CBD),469-479 King Street MELBOURNE 3000,Zoological and Botanical Gardens Operation,Seats - Indoor,113,144.9595,-37.8206
1,2002,Melbourne (CBD),469-479 King Street MELBOURNE 3000,Zoological and Botanical Gardens Operation,Seats - Outdoor,38,144.9595,-37.8206
2,2002,Melbourne (CBD),"Vault 12, 387 Flinders Street MELBOURNE 3000",Other Personal Services n.e.c.,Seats - Indoor,15,144.9623,-37.8195
3,2002,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",Health and Fitness Centres and Gymnasia Operation,Seats - Indoor,20,144.9623,-37.8195
4,2002,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",Health and Fitness Centres and Gymnasia Operation,Seats - Outdoor,18,144.9623,-37.8195
5,2002,Melbourne (CBD),"Shop 17, 207 Flinders Street MELBOURNE 3000",Other Gambling Activities,Seats - Indoor,74,144.9653,-37.8187
6,2002,Melbourne (CBD),"Store 73, 0-0 Flinders Street MELBOURNE 3000",Cafes and Restaurants,Seats - Indoor,32,144.9653,-37.8187
7,2002,Melbourne (CBD),"Tenancy 8, 2 Swanston Street MELBOURNE 3000",Cafes and Restaurants,Seats - Indoor,40,144.9705,-37.8178
8,2002,Melbourne (CBD),"Tenancy 3, 2 Swanston Street MELBOURNE 3000",Cafes and Restaurants,Seats - Indoor,40,144.9705,-37.8178
9,2002,Melbourne (CBD),566 Flinders Street MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,25,144.9553,-37.8207


In [49]:
# reading sql using panda
database_df = pd.read_sql('''SELECT * from melbourne_business
                          WHERE
                          census_year >= 2015
                          AND
                          industry_anzsic4_description = 'Other Gambling Activities'
                          ''',engine)
database_df.head()

,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,2015,Melbourne (CBD),"Shop 17, Flinders Street MELBOURNE 3000",Other Gambling Activities,Seats - Indoor,78,144.9653,-37.8187
1,2015,Melbourne (CBD),"Ground , 117 Lonsdale Street MELBOURNE 3000",Other Gambling Activities,Seats - Indoor,50,144.9689,-37.8107
2,2015,Kensington,Part 500 Epsom Road FLEMINGTON 3031,Other Gambling Activities,Seats - Indoor,100,144.9125,-37.7891
3,2016,Melbourne (CBD),"Shop 17, Flinders Street MELBOURNE 3000",Other Gambling Activities,Seats - Indoor,78,144.9653,-37.8187
4,2016,Melbourne (CBD),"Ground , 117 Lonsdale Street MELBOURNE 3000",Other Gambling Activities,Seats - Indoor,50,144.9689,-37.8107


In [13]:
# convert to geojson
from pandas_geojson import to_geojson
geo_json = to_geojson(df=database_df, lat='y_coordinate', lon='x_coordinate',properties=["census_year", "clue_small_area", 
            "trading_name", "industry_anzsic4_description", "seating_type", "number_of_seats"])
geo_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'census_year': 2002,
    'clue_small_area': 'Melbourne (CBD)',
    'trading_name': '469-479 King Street MELBOURNE 3000',
    'industry_anzsic4_description': 'Zoological and Botanical Gardens Operation',
    'seating_type': 'Seats - Indoor',
    'number_of_seats': 113},
   'geometry': {'type': 'Point', 'coordinates': [144.9595, -37.8206]}},
  {'type': 'Feature',
   'properties': {'census_year': 2002,
    'clue_small_area': 'Melbourne (CBD)',
    'trading_name': '469-479 King Street MELBOURNE 3000',
    'industry_anzsic4_description': 'Zoological and Botanical Gardens Operation',
    'seating_type': 'Seats - Outdoor',
    'number_of_seats': 38},
   'geometry': {'type': 'Point', 'coordinates': [144.9595, -37.8206]}},
  {'type': 'Feature',
   'properties': {'census_year': 2002,
    'clue_small_area': 'Melbourne (CBD)',
    'trading_name': 'Vault 12, 387 Flinders Street MELBOURNE 3000',
    'industry_anzsic4_d